# Supplementary Histograms

Histograms for all data variables from the saved reflectivity dataframe.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

%matplotlib widget

In [ ]:
### Load the processed reflectivity dataframe ###

import os

# If the previous notebook has not been run to get bed-echo power download it from Zenodo
fn = '../../proc/Processed_Reflectivity.csv'
if not os.path.exists(fn):
    os.system('wget -L https://zenodo.org/records/11201199/files/Picked_Bed_Power.csv -O ../../proc/Processed_Reflectivity.csv');
# Read as pandas dataframe
df = pd.read_csv(fn)

In [ ]:
### Function used to make all the histograms below ###

def plot_map_histogram(df,variable,bins1,bins2,vmin,vmax,label=''):
    
    plt.figure(figsize=(10,4))

    ax1 = plt.subplot(121)
    plt.scatter(df['X']/1000.,df['Y']/1000.,c=df[variable],cmap='magma',s=1,vmin=vmin,vmax=vmax)
    plt.axis('equal')
    plt.colorbar(orientation='vertical',label=label)
    plt.xlabel('PS Easting (km)')
    plt.ylabel('PS Northing (km)')
    
    ax2 = plt.subplot(122)
    plt.hist(df[variable],bins=bins1,color='k');
    # Grounded and Not a Lake
    plt.hist(df[(df['Grounded']==True) & (df['Lake']==False)][variable],bins=bins1,color='grey');
    # Floating
    plt.hist(df[df['Grounded']==False][variable],bins=bins1,color='lightgrey');
    # Lakes
    plt.hist(df[df['Lake']==True][variable],bins=bins2,color='skyblue');

    plt.xlim(vmin,vmax)
    plt.ylabel('# Traces')
    plt.xlabel(label)
    plt.tight_layout()
    
    return

In [ ]:
# Raw Picked Power
plot_map_histogram(df,'P_bed',500,100,-155,-75,label='Raw Bed Power (dB)')
plt.savefig('fig-S1-1.png',dpi=300)

In [ ]:
# Raw Picked Power of surface reflection
plot_map_histogram(df,'P_surf',500,100,-110,-60,label='Raw Surface Power (dB)')
plt.savefig('fig-S1-2.png',dpi=300)

In [ ]:
# Crossover Adjustments
df['M'] = df['P_bed_adj'] - df['P_bed']
plot_map_histogram(df,'M',35,35,-20,20,'Crossover Adjustment (dB)')
plt.savefig('fig-S1-3.png',dpi=300)

In [ ]:
# Aircraft height above ice
plot_map_histogram(df,'h',500,100,100,2100,'Aircraft Height (m)')
plt.savefig('fig-S1-4.png',dpi=300)
# Ice Thickness
plot_map_histogram(df,'H',500,100,400,900,'Ice Thickness (m)')
plt.savefig('fig-S1-5.png',dpi=300)

In [ ]:
# Relative Reflectivity for 3 different attenuation rates

epsr_ice = 3.15

# 5 dB/km
label = 'R5'
G = 20*np.log10(2.*(df['h'] + df['H']/np.sqrt(epsr_ice)))
N = 5   # Attenuation
L = 2.*N * df['H']/1000.   # one-way rate (dB/km)
df[label] = df['P_bed_adj'] + G + L  # Reflectivity
df[label] -= df[label].mean()  # Move the mean to 0
plot_map_histogram(df,label,500,100,-30,30,'[R] (with 5 dB/km attenuation)')
plt.savefig('fig-S1-6.png',dpi=300)

# 10 dB/km
label = 'R10'
G = 20*np.log10(2.*(df['h'] + df['H']/np.sqrt(epsr_ice)))
N = 10   # Attenuation
L = 2.*N * df['H']/1000.   # one-way rate (dB/km)
df[label] = df['P_bed_adj'] + G + L  # Reflectivity
df[label] -= df[label].mean()  # Move the mean to 0
plot_map_histogram(df,label,500,100,-30,30,'[R] (with 10 dB/km attenuation)')
plt.savefig('fig-S1-7.png',dpi=300)

# 15 dB/km
label = 'R15'
G = 20*np.log10(2.*(df['h'] + df['H']/np.sqrt(epsr_ice)))
N = 15   # Attenuation
L = 2.*N * df['H']/1000.   # one-way rate (dB/km)
df[label] = df['P_bed_adj'] + G + L  # Reflectivity
df[label] -= df[label].mean()  # Move the mean to 0
plot_map_histogram(df,label,500,100,-30,30,'[R] (with 15 dB/km attenuation)')
plt.savefig('fig-S1-8.png',dpi=300)

In [ ]:
### Calculate RMS height of ice surface and bed ###

df['S'] = df['Elevation'] - df['h']
df['B'] = df['S'] - df['H']

df['RMS_S'] = np.nan*df['X']
df['RMS_B'] = np.nan*df['X']
c = 3e8
epsr_ice = 3.15
c_ice = c/np.sqrt(epsr_ice)
freq = 300e6
lam_air = c/freq
lam_ice = c/freq/np.sqrt(epsr_ice)

def rms_height(h):
    h_ = h - np.nanmean(h)
    rms_h = np.sqrt(np.nanmean(h_**2.))
    return rms_h

def moving_rms_height(H,win):
    out = np.nan*np.ones_like(H)
    for i in range(len(H)):
        h = H[max(0,i-win//2):min(len(H),i+win//2)]
        out[i] = rms_height(h)
    return out

fdays = df['Flight Day'].unique()
for d in fdays:
    print('Flight Day',d)
    segs = df[df['Flight Day']==d]['Segment'].unique()
    for s in segs:
        print('Segment',s)
        frames = df[(df['Flight Day']==d) & (df['Segment']==s)]['Frame'].unique()
        for f in frames:
            idxs = (df['Flight Day']==d) & (df['Segment']==s) & (df['Frame']==f)

            h = np.nanmean(df.loc[idxs,'h'])
            H = np.nanmean(df.loc[idxs,'H'])
            Dsurf = np.sqrt(2.*lam_air*h)
            Dbed = np.sqrt(2.*(lam_air*h+lam_ice*H/np.sqrt(epsr_ice)))
            x = df.loc[idxs,'X'].values
            y = df.loc[idxs,'Y'].values
            dist = np.sqrt((x[1:]-x[:-1])**2.+(y[1:]-y[:-1])**2.)
            dx = np.nanmedian(dist)
            
            bin_size = int(max(2,Dsurf//dx))
            df.loc[idxs,'RMS_S'] = moving_rms_height(df['S'][idxs].values,bin_size)
            bin_size = int(max(2,Dbed//dx))
            df.loc[idxs,'RMS_B'] = moving_rms_height(df['B'][idxs].values,bin_size)

In [ ]:
# Surface Elevation
plot_map_histogram(df,'S',500,100,0,120,'Surface Elevation (m)')
plt.savefig('fig-S1-9.png',dpi=300)
# Surface RMS Height
plot_map_histogram(df,'RMS_S',5000,1000,0,.05,'RMS Surface Height (m)')
plt.savefig('fig-S1-10.png',dpi=300)

In [ ]:
# Bed Elevation
plot_map_histogram(df,'B',500,100,-800,-300,'Bed Elevation (m)')
plt.savefig('fig-S1-11.png',dpi=300)
# Bed RMS Height
plot_map_histogram(df,'RMS_B',5000,1000,0,.5,'RMS Bed Height (m)')
plt.savefig('fig-S1-12.png',dpi=300)